## Limpando e manipulando dados após Web Scraping

In [1]:
import pandas as pd

In [2]:
import re

In [3]:
file = "datasets/imoveis.json"

df = pd.read_json(file, orient = 'records')

df.head()

,preco,dormitorios,area,vagas,bairro
0,[R$ 1.055.000],[3],[128],[3],"[Imóvel 106391 - , Apartamento, \n ..."
1,[R$ 1.112.000],[3],[136.34],[2],"[Imóvel 130624 - , Apartamento, \n ..."
2,[R$ 1.620.000],[3],[204.87],[3],"[Imóvel 192740 - , Apartamento, \n ..."
3,[R$ 1.190.000],[3],[241],[2],"[Imóvel 196741 - , Casa, \n ..."
4,[R$ 3.053.011],[4],[271.3],[4],"[Imóvel 183630 - , Apartamento, \n ..."


In [4]:
print('Dimensões do Dataframe:\n')
print(f'Linhas: {df.shape[0]}')
print(f'Colunas: {df.shape[1]}')

Dimensões do Dataframe:

Linhas: 600
Colunas: 5


In [5]:
df.dtypes

preco          object
dormitorios    object
area           object
vagas          object
bairro         object
dtype: object

In [6]:
df['preco']

0      [R$ 1.055.000]
1      [R$ 1.112.000]
2      [R$ 1.620.000]
3      [R$ 1.190.000]
4      [R$ 3.053.011]
            ...      
595      [R$ 860.000]
596      [R$ 325.000]
597    [R$ 1.800.000]
598    [R$ 1.750.000]
599    [R$ 1.380.000]
Name: preco, Length: 600, dtype: object

In [7]:
df['preco'].sample(20)

388      [R$ 355.000]
589      [R$ 875.000]
499      [R$ 650.000]
318      [R$ 875.000]
66       [R$ 790.000]
100      [R$ 310.000]
113      [R$ 890.000]
289      [R$ 185.000]
545      [R$ 340.000]
416      [R$ 248.000]
521      [R$ 490.000]
394      [R$ 498.000]
88     [R$ 1.699.000]
529      [R$ 650.000]
264      [R$ 223.500]
518      [R$ 530.000]
327      [R$ 480.000]
564      [R$ 424.000]
171      [R$ 934.065]
509      [R$ 730.000]
Name: preco, dtype: object

In [8]:
df['preco'] = df['preco'].apply(lambda x : re.sub('\D', '', str(x))).astype(float)

In [9]:
df['preco']

0      1055000.0
1      1112000.0
2      1620000.0
3      1190000.0
4      3053011.0
         ...    
595     860000.0
596     325000.0
597    1800000.0
598    1750000.0
599    1380000.0
Name: preco, Length: 600, dtype: float64

In [10]:
df['preco'].sample(10)

481     799000.0
236    1210000.0
127    1819000.0
360     315000.0
194     478703.0
539     420000.0
564     424000.0
435     704499.0
178     630000.0
469     640000.0
Name: preco, dtype: float64

In [11]:
df.dtypes

preco          float64
dormitorios     object
area            object
vagas           object
bairro          object
dtype: object

In [12]:
df.isnull().sum()

preco           0
dormitorios     8
area            0
vagas          21
bairro          0
dtype: int64

In [13]:
df['dormitorios'].fillna("0", inplace = True)

In [14]:
df['dormitorios'] = df['dormitorios'].apply(lambda x : re.sub('\D', '', str(x))).astype(int)

In [15]:
df['dormitorios'].value_counts()

3    319
2    184
1     52
4     34
0      8
6      2
5      1
Name: dormitorios, dtype: int64

In [16]:
df['area'] = df['area'].apply(lambda x : re.sub('\[', '', str(x)))
df['area'] = df['area'].apply(lambda x : re.sub('\]', '', str(x)))
df['area'] = df['area'].apply(lambda x : re.sub("'", '', str(x)))
df['area'] = df['area'].apply(lambda x : re.sub("'", '', str(x))).astype(float)

In [17]:
df['area'].sample(20)

188    519.54
454     60.37
197     74.23
204     80.41
8       86.20
503     66.51
315     48.46
125     87.86
110    192.69
457     72.69
482    100.00
191    250.62
247     59.93
357    105.61
143    139.65
217     69.29
461    221.33
35      36.76
194     67.00
387    199.88
Name: area, dtype: float64

In [18]:
df.dtypes

preco          float64
dormitorios      int32
area           float64
vagas           object
bairro          object
dtype: object

In [19]:
df['vagas'].fillna("0", inplace = True)

In [20]:
df['vagas'] = df['vagas'].apply(lambda x : re.sub('\D', '', str(x))).astype(int)

In [21]:
df['vagas'].value_counts()

1    264
2    231
3     45
4     30
0     21
6      6
5      2
8      1
Name: vagas, dtype: int64

In [22]:
df['bairro'].sample(10)

45     [Imóvel 199151 - , Apartamento, \n            ...
508    [Imóvel 180878 - , Apartamento, \n            ...
148    [Imóvel 123696 - , Apartamento, \n            ...
485    [Imóvel 166098 - , Apartamento, \n            ...
419    [Imóvel 199105 - , Apartamento, \n            ...
230    [Imóvel 178174 - , Cobertura, \n              ...
363    [Imóvel 165242 - , Apartamento, \n            ...
544    [Imóvel 190325 - , Apartamento, \n            ...
536    [Imóvel 189226 - , Casa, \n                   ...
573    [Imóvel 195168 - , Apartamento, \n            ...
Name: bairro, dtype: object

In [23]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('\d', '', str(x))).astype(str)

In [24]:
df['bairro'].sample(10)

362    ['Imóvel  - ', 'Apartamento', '\n             ...
367    ['Imóvel  - ', 'Apartamento', '\n             ...
96     ['Imóvel  - ', 'Apartamento', '\n             ...
175    ['Imóvel  - ', 'Cobertura', '\n               ...
193    ['Imóvel  - ', 'Apartamento', '\n             ...
553    ['Imóvel  - ', 'Apartamento', '\n             ...
219    ['Imóvel  - ', 'Casa', '\n                    ...
29     ['Imóvel  - ', 'Apartamento', '\n             ...
165    ['Imóvel  - ', 'Apartamento', '\n             ...
220    ['Imóvel  - ', 'Apartamento', '\n             ...
Name: bairro, dtype: object

In [25]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('Imóvel  -', '', x))

In [26]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub(r'\\n', '', x))

In [27]:
df['bairro'].sample(10)

388    [' ', 'Apartamento', '                        ...
581    [' ', 'Apartamento', '                        ...
86     [' ', 'Apartamento', '                        ...
70     [' ', 'Casa', '                        Belém N...
0      [' ', 'Apartamento', '                        ...
205    [' ', 'Casa', '                        Cavalha...
528    [' ', 'Apartamento', '                        ...
46     [' ', 'Apartamento', '                        ...
79     [' ', 'Apartamento', '                        ...
536    [' ', 'Casa', '                        Jardim ...
Name: bairro, dtype: object

In [28]:
df['tipo'] = df['bairro'].apply(lambda x : re.split(',', x)[1])

In [29]:
df['tipo'].sample(10)

333     'Apartamento'
557     'Apartamento'
411     'Apartamento'
374     'Apartamento'
376            'Casa'
347     'Apartamento'
431     'Apartamento'
358            'Sala'
340            'Casa'
252       'Cobertura'
Name: tipo, dtype: object

In [30]:
df['bairro'] = df['bairro'].apply(lambda x : re.split(',', x)[2])

In [31]:
df['bairro']

0       '                        Central Parque
1          '                        Menino Deus
2           '                        Rio Branco
3              '                        Sarandi
4           '                        Bela Vista
                         ...                   
595        '                        Auxiliadora
596           '                        Tristeza
597         '                        Bela Vista
598      '                        Jardim Europa
599     '                        Três Figueiras
Name: bairro, Length: 600, dtype: object

In [32]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('\s', '', x))

In [33]:
df['bairro']

0      'CentralParque
1         'MeninoDeus
2          'RioBranco
3            'Sarandi
4          'BelaVista
            ...      
595      'Auxiliadora
596         'Tristeza
597        'BelaVista
598     'JardimEuropa
599    'TrêsFigueiras
Name: bairro, Length: 600, dtype: object

In [34]:
df['tipo'] = df['tipo'].apply(lambda x : re.sub("'", "", x))
df['bairro'] = df['bairro'].apply(lambda x : re.sub("'", '', x))

In [35]:
df['tipo'].head()

0     Apartamento
1     Apartamento
2     Apartamento
3            Casa
4     Apartamento
Name: tipo, dtype: object

In [36]:
df['bairro'].head()

0    CentralParque
1       MeninoDeus
2        RioBranco
3          Sarandi
4        BelaVista
Name: bairro, dtype: object

In [37]:
df.nunique()

preco          366
dormitorios      7
area           490
vagas            8
bairro          72
tipo             8
dtype: int64

In [38]:
df['tipo'].value_counts()

 Apartamento    481
 Casa            73
 Cobertura       27
 Loft            10
 Sala             6
 Loja             1
 Terreno          1
 Flat             1
Name: tipo, dtype: int64

In [39]:
df['preco'] = df['preco'].apply(lambda x : round(x))
df['preco']

0      1055000
1      1112000
2      1620000
3      1190000
4      3053011
        ...   
595     860000
596     325000
597    1800000
598    1750000
599    1380000
Name: preco, Length: 600, dtype: int64

In [40]:
df['area']

0      128.00
1      136.34
2      204.87
3      241.00
4      271.30
        ...  
595     93.65
596     68.00
597    211.00
598    163.00
599    154.00
Name: area, Length: 600, dtype: float64

In [41]:
df.describe()

,preco,dormitorios,area,vagas
count,6.000000e+02,600.000000,600.000000,600.000000
mean,8.636065e+05,2.550000,117.512045,1.725000
std,9.296298e+05,0.815644,89.498635,1.015428
min,1.178000e+05,0.000000,23.330000,0.000000
25%,3.800000e+05,2.000000,64.000000,1.000000
50%,5.870000e+05,3.000000,84.465000,2.000000
75%,9.710585e+05,3.000000,138.792500,2.000000
max,9.117117e+06,6.000000,751.510000,8.000000


In [42]:
df.groupby(by = 'bairro').preco.mean().sort_values(ascending = False)[:15]

bairro
MontSerrat          2.213700e+06
BelaVista           2.204248e+06
BelémNovo           2.134000e+06
PedraRedonda        1.950000e+06
JardimSãoPedro      1.900000e+06
TrêsFigueiras       1.667200e+06
Auxiliadora         1.588450e+06
JardimEuropa        1.455259e+06
CentralParque       1.244618e+06
MoinhosdeVento      1.242944e+06
ChácaradasPedras    1.221700e+06
RioBranco           1.190297e+06
Petrópolis          1.170219e+06
Agronomia           9.677143e+05
MeninoDeus          9.590551e+05
Name: preco, dtype: float64

In [43]:
df.loc[df['bairro'] == "BelaVista"].sort_values(by = 'preco', ascending = False)

,preco,dormitorios,area,vagas,bairro,tipo
444,9117117,4,509.38,6,BelaVista,Cobertura
188,3896550,3,519.54,4,BelaVista,Apartamento
166,3420000,4,226.02,4,BelaVista,Apartamento
275,3303911,4,257.60,4,BelaVista,Apartamento
268,3240374,4,257.60,4,BelaVista,Apartamento
4,3053011,4,271.30,4,BelaVista,Apartamento
302,2700000,3,257.80,4,BelaVista,Apartamento
441,2490000,3,174.46,3,BelaVista,Apartamento
229,2090000,4,350.71,3,BelaVista,Cobertura
195,2071000,3,163.45,2,BelaVista,Apartamento


In [44]:
df.to_csv('datasets/new_imoveis.csv', index = False)